#Бот Гаврила Новый

#Общая инструкция по созданию и работе с телеграм ботом
Шаг 1. Регистрируем бота

Находим в поиске Телеграма - бота @botFather — это главный инструмент для создания ботов. Набираем в нём /newbot.
Придумываем название и имя боту. Название — то, как он будет подписан в списке контактов. Имя — строка, по которой его можно будет найти. Обратите внимание, что имя должно оканчиваться на bot. В ответ мы получим токен — длинную последовательность символов, которая пригодится нам в будущем. Мы назвали бота echoKeksBot, но вам придётся придумать другое название, потому что это уже занято. Извините.

Шаг 2.

в коде самого ботанаходим строку кода - "token = 'ваш_токен'" и подставляем туда ваш токен.

Шаг 3.

Последовательно запускаем ячейки данного ноутбука

(Код самого бота находится в конце этой тетрадки, именно там нужно записать свой токен в строку кода "token = 'ваш_токен'"
Запуская бота в гугл колабе мы заставляем ячейку с кодом бота выполнять бесконечный цикл. Поэтому ваш бот будет работать пока длится сессия в гугл колабе.)

Шаг 4.

Переходим в телеграм, открываем вашего бота, вводим команду /start

Шаг 5.

Общаемся с ботом

Останавливаем бот повторным нажатием на ячейку с кодом бота.


#Бот в этом ноутбуке имеет следующий функционал:

1) чат с LLM - llama-7B-hf (на английском и русском языках) взято с https://huggingface.co/baffo32/decapoda-research-llama-7B-hf

2)чат с chatgpt 3.5 (используется api сайта openai.com)

3)предсказание температуры воздуха в Москве с выводом графика (интервал прогнозирования с 11.11.2023 до 09.01.2024г.) Сами спронозированные значения температуры находятся здесь: https://drive.google.com/file/d/15Yxkm93tTsgStHPcUU2W5W_yv3GQyuDI/view?usp=sharing

4)Вывод значений температуры воздуха в любом городе на текущую дату (используется api сайта openweathermap.org)

#Подключаем библиотеки для LLM

In [ ]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers

#Скачиваем модель и токенизатор

In [ ]:
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("baffo32/decapoda-research-llama-7B-hf")
model = LLaMAForCausalLM.from_pretrained(
    "baffo32/decapoda-research-llama-7B-hf",
    load_in_8bit=True,
    device_map='auto',
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'LLaMATokenizer'.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


ValueError: ignored

#Устанавливаем переводчик для перевода ответов модели на русский

In [ ]:
!pip install -q googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.2 MB/s eta 0:00:00


In [ ]:
from googletrans import Translator

#Функции перевода и получения ответа модели на русском языке

In [ ]:
translator = Translator()

def translate_text(text):
    translation = translator.translate(text, src='ru', dest='en')
    return translation.text

def translate_text_ru(text):
    translation = translator.translate(text, src='en', dest='ru')
    return translation.text

def generate_prompt(instruction, input=None):
    instruction_english = translate_text(instruction)
    if input:
        input_english = translate_text(input)
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction_english}

### Input:
{input_english}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction_english}

### Response:"""

generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    )

def evaluate(instruction, input=None):
    instruction_english = translate_text(instruction)
    input_english = None
    if input:
        input_english = translate_text(input)
    prompt = generate_prompt(instruction_english, input_english)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=50
    )
    responses = []
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        if "### Response:" in output:
            response_start = output.index("### Response:") + len("### Response:")
            response_end = output.index("\n\n", response_start) if "\n\n" in output[response_start:] else None
            response = output[response_start:response_end].strip()  # get text after "### Response:" and before "\n\n"
            responses.append(response)
            output_russian = translate_text_ru(output.split("### Response:")[1].strip())
        return output_russian

#Функция получения ответа на родном языке модели (английский)

In [ ]:
def evaluate1(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=50
    )
    responses = []
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        if "### Response:" in output:
            response_start = output.index("### Response:") + len("### Response:")
            response_end = output.index("\n\n", response_start) if "\n\n" in output[response_start:] else None
            response = output[response_start:response_end].strip()  # get text after "### Response:" and before "\n\n"
            responses.append(response)
    return responses[0]

#Функцция для получения ответа от chatgpt 3.5

In [ ]:
import requests

def fetch_chatbot_response(query):
    # Ключ из личного кабинета, подставьте свой
    CHAD_API_KEY = 'Здесь ваш ключ'

    # Формируем запрос
    request_json = {
        "message": query,
        "api_key": CHAD_API_KEY,
    }

    try:
        # Отправляем запрос и дожидаемся ответа
        response = requests.post(url='https://ask.chadgpt.ru/api/public/gpt-3.5',
                                 json=request_json)

        # Проверяем, отправился ли запрос
        if response.status_code != 200:
            return f'Ошибка! Код http-ответа: {response.status_code}'
        else:
            # Получаем текст ответа и преобразовываем в dict
            resp_json = response.json()

            # Если успешен ответ, то выводим
            if resp_json['is_success']:
                resp_msg = resp_json['response']
                used_words = resp_json['used_words_count']
                return f'Ответ от бота: {resp_msg}'
            else:
                error = resp_json['error_message']
                return f'Ошибка: {error}'
    except requests.exceptions.RequestException as err:
        print (f"Возникла ошибка: {err}")

#Устанавливаем api Telegram

In [ ]:
!pip install pytelegrambotapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.14.0-py3-none-any.whl size=215250 sha256=e2c777fe3293954af52471d098893aaa4be4889ba89faecedfb73ee44c96c61d
  Stored in directory: /root/.cache/pip/wheels/25/51/2d/24b40a366c85c37928d5aa36ddf257e5a79fad25e1ecd11b2c
Successfully built pytelegrambotapi


#Импортируем библиотеки

In [ ]:
import requests

In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import io

#Внимание!!! Перед запуском бота нужно загрузить в сессионное хранилище файл с прогнозными значениями - "df_all.csv"

https://drive.google.com/uc?id=15Yxkm93tTsgStHPcUU2W5W_yv3GQyuDI

In [ ]:
!gdown 15Yxkm93tTsgStHPcUU2W5W_yv3GQyuDI

Downloading...
From: https://drive.google.com/uc?id=15Yxkm93tTsgStHPcUU2W5W_yv3GQyuDI
To: /content/df_all.csv
100% 5.15k/5.15k [00:00<00:00, 23.6MB/s]


#Код самого бота

In [ ]:
import telebot
from telebot import types

token = 'Здесь ваш токен'
bot = telebot.TeleBot(token)

date_requested = False  # Переменная для отслеживания режима предсказания погоды
ctrl_button = False
lora_eng = False
lora_rus = False
gpt_ok = False
forecast_requested = False
img_ok = True

# Главное меню с кнопками
@bot.message_handler(commands=['start'])
def handle_start(message):
    markup = types.ReplyKeyboardMarkup(row_width=2)
    itembtn1 = types.KeyboardButton('погода')
    itembtn2 = types.KeyboardButton('Вопрос LLM')
    markup.add(itembtn1, itembtn2)
    bot.send_message(chat_id=message.chat.id, text="Выберите один из вариантов:", reply_markup=markup)


# Обработчик кнопки "погода"
@bot.message_handler(func=lambda message: message.text == 'погода')
def handle_weather(message):
    markup = types.InlineKeyboardMarkup(row_width=2)
    itembtn1 = types.InlineKeyboardButton('сегодня', callback_data='today')
    itembtn2 = types.InlineKeyboardButton('прогноз для Москвы', callback_data='forecast')
    markup.add(itembtn1, itembtn2)
    global ctrl_button
    ctrl_button=True
    bot.send_message(chat_id=message.chat.id, text="Выберите один из вариантов:", reply_markup=markup)

# Обработчик кнопки "Вопрос LLM"
@bot.message_handler(func=lambda message: message.text == 'Вопрос LLM')
def handle_lora(message):
    markup = types.InlineKeyboardMarkup(row_width=2)
    itembtn1 = types.InlineKeyboardButton('LLM русский', callback_data='russian')
    itembtn2 = types.InlineKeyboardButton('LLM английский', callback_data='english')
    itembtn3 = types.InlineKeyboardButton('gpt-3.5', callback_data='gpt')
    markup.add(itembtn1, itembtn2, itembtn3)
    global ctrl_button
    ctrl_button=True
    bot.send_message(chat_id=message.chat.id, text="Выберите язык:", reply_markup=markup)


# Обработчик нажатий инлайн-кнопок
@bot.callback_query_handler(func=lambda call: True)
def handle_inline_buttons(call):
    if call.data == 'today':
        bot.send_message(chat_id=call.message.chat.id, text="Введите название города")
        global date_requested
        global forecast_requested
        date_requested = False
        forecast_requested = True
    elif call.data == 'forecast':
        bot.send_message(chat_id=call.message.chat.id, text="Введите дату для прогноза погоды в Москве \n (в формате YYYY-MM-DD)")
        #global date_requested
        date_requested = True
        forecast_requested = False
    elif call.data == 'russian':
        bot.send_message(chat_id=call.message.chat.id, text="Привет, я - LLM!\n Общайтесь со мной на русском, но умоляю - не заставляйте писать код!")
        global lora_eng
        global lora_rus
        lora_eng = False
        lora_rus = True
        date_requested = False
        forecast_requested = False
    elif call.data == 'english':
        bot.send_message(chat_id=call.message.chat.id, text="Hello, I'm LLM! Ask me anything")
        lora_eng = True
        lora_rus = False
        date_requested = False
        forecast_requested = False
    elif call.data == 'gpt':
        bot.send_message(chat_id=call.message.chat.id, text="Привет, я chatgpt! Спроси меня о чем нибудь!")
        lora_eng = False
        lora_rus = False
        date_requested = False
        forecast_requested = False
        global gpt_ok
        gpt_ok = True
    else:
        bot.send_message(chat_id=call.message.chat.id, text="Такого варианта нет")

# Обработчик ввода названия города
@bot.message_handler(func=lambda message: forecast_requested == True and ctrl_button==True)
def handle_city(message):
    city = message.text
    o=f1(city)  # Вызов функции для получения температуры воздуха в указанном городе
    bot.send_message(chat_id=message.chat.id, text=o)

# Обработчик ввода даты для прогноза погоды
@bot.message_handler(func=lambda message: date_requested == True and ctrl_button==True)
def handle_date(message):
    date = message.text
    o1=f2(date)  # Вызов функции для отображения графика погоды на указанную дату
    #bot.send_message(chat_id=message.chat.id, text=o1)
    global img_ok
    if img_ok == True:
        with open('forecast.png', 'rb') as photo:
            bot.send_photo(chat_id=message.chat.id, photo=photo)
    bot.send_message(chat_id=message.chat.id, text=o1)

# Обработчик ввода сообщения для Лоры английской
@bot.message_handler(func=lambda message: lora_eng == True and ctrl_button==True)
def process_message_eng(message):
    user_message_eng = message.text
    answer1 = lora_eng_f(user_message_eng)
    bot.send_message(chat_id=message.chat.id, text=answer1)

# Обработчик ввода сообщения для Лоры русской
@bot.message_handler(func=lambda message: lora_rus == True and ctrl_button==True)
def process_message_rus(message):
    user_message_rus = message.text
    answer2 = lora_rus_f(user_message_rus)
    bot.send_message(chat_id=message.chat.id, text=answer2)

# Обработчик ввода сообщения для chatgpt
@bot.message_handler(func=lambda message: gpt_ok == True and ctrl_button==True)
def process_message_gpt(message):
    user_message_gpt = message.text
    answer2 = fetch_chatbot_response(user_message_gpt)
    bot.send_message(chat_id=message.chat.id, text=answer2)

# Обработчик всех остальных текстовых сообщений
@bot.message_handler(func=lambda m: True)  # этот обработчик должен идти последним
def echo_all(message):
    bot.reply_to(message, 'Я не понимаю')

# Функция для обработки сообщений пользователя Лорой английской
def lora_eng_f(message):
    # Мой код для обработки сообщений пользователя Лорой
    result = evaluate1(message)
    #result ="Hello, i'm Lora"
    return result

# Функция для обработки сообщений пользователя Лорой русской
def lora_rus_f(message):
    # Мой код для обработки сообщений пользователя Лорой
    result = evaluate(message)
    #result ="Привет, я Лора"
    return result

# Функция для получения температуры воздуха в указанном городе
def f1(city):
    import requests
    url = 'https://api.openweathermap.org/data/2.5/weather?q='+city+'&units=metric&lang=ru&appid=79d1ca96933b0328e1c7e3e7a26cb347'

    try:
        # Отправляем запрос на сервер и получаем результат
        weather_data = requests.get(url).json()
        # Получаем данные о температуре и ощущается ли она
        temperature = round(weather_data['main']['temp'])
        temperature_feels = round(weather_data['main']['feels_like'])
        return f"Температура в городе {city} сейчас {str(temperature)} °C \n Ощущается как {str(temperature_feels)} °C."
    except:
        return "Города не существует"

# Функция для отображения графика погоды на указанную дату
def f2(date):
    global img_ok
    # Здесь должна быть логика получения графика погоды на указанную дату
    from datetime import datetime
    # Проверяем, что дата в правильном формате
    try:
       valid_date = datetime.strptime(date, '%Y-%m-%d')
    except ValueError:
        img_ok = False
        return("Неверный формат даты. Пожалуйста, введите дату в формате YYYY-MM-DD")
    df_all = pd.read_csv('df_all.csv')
    df_all['date'] = pd.to_datetime(df_all['date'])
    # Проверяем, что дата находится в диапазоне дат в DataFrame
    if 'date' in df_all:
        min_date = df_all['date'].min()
        max_date = df_all['date'].max()

        if valid_date < min_date or valid_date > max_date:
            img_ok = False
            return("Введенная дата выходит за границы  периода прогноза.")
    # Фильтруем исходный DataFrame по дате
    date_filter = df_all['date'] >= pd.to_datetime(date)
    df_filtered = df_all.loc[date_filter]

    plt.figure(figsize=(20, 6))

    # Строим графики с использованием отфильтрованного DataFrame
    plt.plot(df_filtered['date'], df_filtered['y_lstm'], label='Прогноз LSTM')
    plt.plot(df_filtered['date'], df_filtered['y_prophet2'], label='Прогноз Prophet')
    plt.plot(df_filtered['date'], df_filtered['y_fourier2'], label='Прогноз Fourier')
    plt.plot(df_filtered['date'], df_filtered['y_mean'], label='Среднее значение')

    # Указываем названия осей и заголовок
    plt.title('Прогноз Температуры воздуха начиная с ' + date)
    plt.xlabel('Дата')
    plt.ylabel('Температура')

    # Конвертация времени для удобного отображения на графике
    plt.gcf().autofmt_xdate()

    # Показываем легенду
    plt.legend()

    # Сохранение графика в файл
    plt.savefig('forecast.png')
    plt.close()
    #plt.show()
    val_lstm = df_all[df_all['date'] == date]['y_lstm'].values[0]
    val_prophet = df_all[df_all['date'] == date]['y_prophet2'].values[0]
    val_fourier = df_all[df_all['date'] == date]['y_fourier2'].values[0]
    val_mean = df_all[df_all['date'] == date]['y_mean'].values[0]
    # Например: bot.send_photo(chat_id=call.message.chat.id, photo=open('path_to_image', 'rb'))
    # Ваш код для предсказания погоды на указанную дату
    # Отправка сообщения с текстом и изображением
    img_ok = True
    return (
    f'Прогноз от LSTM на {date}\n температура воздуха составит {val_lstm:.1f} °C.\n\n'
    f'Прогноз от prophet на {date}\n температура воздуха составит {val_prophet:.1f} °C.\n\n'
    f'Прогноз от Fourier на {date}\n температура воздуха составит {val_fourier:.1f} °C.\n\n'
    f'Прогноз от mean на {date}\n температура воздуха составит {val_mean:.1f} °C.')


bot.polling()